In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pets = pd.read_csv("data/dummy_pets.csv")
procedures = pd.read_csv("data/dummy_procedures.csv")

In [ ]:
pets.head()
# pets.columns

procedures.head()
# procedures.columns

In [ ]:
pets["age"].isnull().value_counts()
procedures["procedure_code"].isnull().value_counts() # This column has null values TODO
procedures["week_start_date"].isnull().value_counts()

In [ ]:
df = procedures[procedures["procedure_code"].isnull()]
df["description"].value_counts()
# C-SEC
# VIRTUAL BRAVECTO 9-22
procedures[procedures["description"]=="C-SEC"]
procedures[procedures["description"]=="VIRTUAL BRAVECTO 9-22"]
procedures[["description","procedure_code"]].value_counts()

# As the number of rows with null values in procecure code is 256, given total of 607262 i.e 0.042%, dropping the null values might affect the analysis as we are trying to fing maximum is various fields, where count is more close giving the maximum
# Hence instead of dropping the rows, we add a temp procedure code for "C-SEC" as CSEC and "VIRTUAL BRAVECTO 9-22" as VIRTBRAV

procedures[procedures["description"]=="C-SEC"]
procedures[procedures["description"]=="VIRTUAL BRAVECTO 9-22"]

# df.loc[df['c1'] == 'Value', 'c2'] = 10 referenced from stack overflow
procedures.loc[procedures["description"]=="C-SEC", "procedure_code"] = "CSEC"
procedures.loc[procedures["description"]=="VIRTUAL BRAVECTO 9-22", "procedure_code"] = "VIRTBRAV"

In [ ]:
procedures[procedures["procedure_code"] == "CSEC"].value_counts()
procedures[procedures["procedure_code"] == "VIRTBRAV"].value_counts()
# Now we have replaced the null values and have a clean dataset of pets and procedures

In [ ]:
# df = procedures.copy()
# df['week_start_date'] = pd.to_datetime(df['week_start_date'], format='%m/%d/%y')
# df['year'] = df['week_start_date'].dt.year
# df.head()
procedures['week_start_date'] = pd.to_datetime(procedures['week_start_date'], format='%m/%d/%y')
procedures['year'] = procedures['week_start_date'].dt.year
procedures.head()


### Tasks

### Practice Analysis 

o Count of unique companies.

o Count of unique practice locations.

o Find the practice with the most procedures in 2022, and plot their procedure counts 
over all years compared to the average of all practices.

o Find the practice with the most production in 2017, and plot their production totals 
over all years compared to the average of all practices.

In [ ]:
unique_companies = procedures['companyid'].nunique()
print(unique_companies)

In [ ]:
unique_practice_locations = procedures[['companyid', 'locationid']].drop_duplicates().shape[0]
print(unique_practice_locations)

In [ ]:
procedures_2022 = procedures[procedures['year']==2022]
procedures_2022.head()
practice_most_procedures_2022 = procedures_2022.groupby(['companyid', 'locationid'])
practice_most_procedures_2022 = practice_most_procedures_2022.size().idxmax()
print(practice_most_procedures_2022)
# practice_most_procedures_2022[0]
# practice_most_procedures_2022[1]

In [ ]:
practice_procedures = procedures[(procedures['companyid'] == practice_most_procedures_2022[0]) & (procedures['locationid'] == practice_most_procedures_2022[1])]
yearly_counts = practice_procedures.groupby(["year"]).size()
print(yearly_counts)

avg_yearly_counts = procedures.groupby(["year", "companyid", "locationid"]).size().groupby(["year"]).mean()
print(avg_yearly_counts)

plt.plot(yearly_counts.index, yearly_counts.values, label='Most Practice Procedures 22\'')
plt.plot(avg_yearly_counts.index, avg_yearly_counts.values, label='Average Procedures',  linestyle='--')
plt.xlabel('Year')
plt.ylabel('Procedure Counts')
plt.title('Procedure Counts Over Years')
plt.legend()
plt.show()

In [ ]:
procedures_2017 = procedures[procedures["year"]==2017]
practice_most_production_2017 = procedures_2017.groupby(['companyid', 'locationid'])['production'].sum().idxmax()
print(practice_most_production_2017)

In [ ]:
practice_production = procedures[(procedures['companyid'] == practice_most_production_2017[0]) & (procedures['locationid'] == practice_most_production_2017[1])]
yearly_production = practice_production.groupby(["year"])['production'].sum()
# yearly_production

avg_yearly_production = procedures.groupby(["year","companyid", "locationid"])['production'].sum().groupby(["year"]).mean()
# avg_yearly_production

plt.plot(yearly_production.index, yearly_production.values, label='Most Production Practice 17\'')
plt.plot(avg_yearly_production.index, avg_yearly_production.values, label='Average Production',  linestyle='--')
plt.xlabel('Year')
plt.ylabel('Production Totals')
plt.title('Production Totals Over Years')
plt.legend()
plt.show()

##### Patient analysis

o Number of unique pets in our database.

o Average number of pets per owner.

o Find the owner with the most pets, and provide a table with details on each pet.

o Find the owner with the most clinic visits (number of procedures), and provide a table 
containing details of all of that owner’s visits.

o Provide analysis of Cats vs. Dogs based on overall procedure history.

In [ ]:
unique_pets = pets['patientid'].nunique()
print(unique_pets)

In [ ]:
avg_pets_per_owner = pets.groupby('clientid').size().mean()
print(int(avg_pets_per_owner))

In [ ]:
owner_most_pets = pets.groupby('clientid').size().idxmax()
owner_most_pets_details = pets[pets['clientid'] == owner_most_pets]
print(owner_most_pets)
owner_most_pets_details

In [ ]:
# Performing similar ops like SQL joins using merge method of pandas dataframe in Python's Pandas
procedures_with_client = procedures.merge(pets[['patientid', 'clientid']], on='patientid')
owner_most_visits = procedures_with_client.groupby('clientid').size().idxmax()
print(owner_most_visits)

In [ ]:
owner_most_visits_details = procedures_with_client[procedures_with_client['clientid'] == owner_most_visits]
df = owner_most_visits_details.copy()
owner_most_visits_details

In [ ]:
# Owner with most pets has most visits LOL
most_visited_owner = pd.merge(procedures_with_client[procedures_with_client['clientid'] == owner_most_visits][["clientid", "companyid", "locationid", "procedure_code"]], pets[["breed", "species", "age", "clientid"]], on='clientid')

In [ ]:
most_visited_owner.value_counts()

In [ ]:
procedures_with_species = procedures.merge(pets[['companyid', 'locationid', 'patientid', 'species']], on=['companyid', 'locationid', 'patientid'])
# procedures_with_species["species"].value_counts()
cats_procedures = procedures_with_species[procedures_with_species['species'] == 'CAT']
dogs_procedures = procedures_with_species[procedures_with_species['species'] == 'DOG']
# dogs_procedures
# cats_procedures

In [ ]:
# Analysis (e.g., counts, common procedures)
cats_procedure_counts = cats_procedures['procedure_code'].value_counts()
dogs_procedure_counts = dogs_procedures['procedure_code'].value_counts()

# Display the results
print("Cats Procedures Counts:")
print(cats_procedure_counts)

print("\nDogs Procedures Counts:")
print(dogs_procedure_counts)

##### Procedure Analysis

o Plot the 10 most common procedure codes by count.

o Provide analysis on the most common procedure code.

o Build a procedure dataset for all procedures containing keyword “IDEXX” and provide 
analysis on this subset.

In [ ]:
top_10_procedures = procedures['procedure_code'].value_counts().head(10)
print(top_10_procedures)
top_10_procedures.plot(kind='bar')
plt.xlabel('Procedure Code')
plt.ylabel('Count')
plt.title('Top 10 Most Common Procedures')
plt.show()

In [ ]:
most_common_procedure_code = procedures['procedure_code'].value_counts().idxmax()
print(most_common_procedure_code)
most_common_procedure_details = procedures[procedures['procedure_code'] == most_common_procedure_code]
most_common_procedure_details

In [ ]:
idexx_procedures = procedures[procedures['description'].str.contains('IDEXX')]
idexx_procedures